# **End of week 1 exercise**

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

## **1. Get a response from your favorite AI Tutor** 

In [1]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [ ]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai = OpenAI()

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [38]:
system_prompt = """You are the software engnieer, phd in mathematics, machine learning engnieer, and other topics"""
system_prompt += """
When responding, always use Markdown for formatting. For any code, use well-structured code blocks with syntax highlighting,
For instance:
```python

sample_list = [for i in range(10)]
```
Another example
```javascript
        function displayMessage() {
            alert("Hello, welcome to JavaScript!");
        }

```

Break down explanations into clear, numbered steps for better understanding. 
Highlight important terms using inline code formatting (e.g., `function_name`, `variable`).
Provide examples for any concepts and ensure all examples are concise, clear, and relevant.
Your goal is to create visually appealing, easy-to-read, and informative responses.

"""


In [39]:
def tutor_user_prompt(question):
    # Ensure the question is properly appended to the user prompt.
    user_prompt = (
        "Please carefully explain the following question in a step-by-step manner for clarity:\n\n"
    )
    user_prompt += question
    return user_prompt

In [43]:


def askTutor(question, MODEL):
    # Generate the user prompt dynamically.
    user_prompt = tutor_user_prompt(question)
    
    # OpenAI API call to generate response.
    if MODEL == 'gpt-4o-mini':
        print(f'You are getting response from {MODEL}')
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )
    else:
        MODEL == 'llama3.2'
        print(f'You are getting response from {MODEL}')
        stream = ollama_via_openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )

    # Initialize variables for response processing.
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    # Process the response stream and update display dynamically.
    for chunk in stream:
        # Safely access the content attribute.
        response_chunk = getattr(chunk.choices[0].delta, "content", "")
        if response_chunk:  # Check if response_chunk is not None or empty
            response += response_chunk
            # No replacement of Markdown formatting here!
            update_display(Markdown(response), display_id=display_handle.display_id)


In [44]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
askTutor(question=question, MODEL=MODEL_GPT)

## **2. Using both LLMs collaboratively approach**

- I thought about like similar the idea of a RAG (Retrieval-Augmented Generation) approach, is an excellent idea to improve responses by refining the user query and producing a polished, detailed final answer. Two LLM talking each other its cool!!! Here's how we can implement this:

**Updated Concept:**
1. Refine Query with Ollama:
    - Use Ollama to refine the raw user query into a well-structured prompt.
    - This is especially helpful when users input vague or poorly structured queries.
2. Generate Final Response with GPT:
    - Pass the refined prompt from Ollama to GPT to generate the final, detailed, and polished response.
3. Return the Combined Output:
    - Combine the input, refined query, and the final response into a single display to ensure clarity.

In [59]:
def refine_with_ollama(raw_question):
    """
    Use Ollama to refine the user's raw question into a well-structured prompt.
    """
    print("Refining the query using Ollama...")
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Refine and structure the following user input."},

        {"role": "user", "content": raw_question},
    ]
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages,
        stream=False  # Non-streamed refinement
    )
    refined_query = response.choices[0].message.content
    return refined_query

In [60]:
def ask_with_ollama_and_gpt(raw_question):
    """
    Use Ollama to refine the user query and GPT to generate the final response.
    """
    # Step 1: Refine the query using Ollama
    refined_query = refine_with_ollama(raw_question)
    
    # Step 2: Generate final response with GPT
    print("Generating the final response using GPT...")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": refined_query},
    ]
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True  # Stream response for dynamic display
    )

    # Step 3: Combine responses
    response = ""
    display_handle = display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:"), display_id=True)
    for chunk in stream:
        response_chunk = getattr(chunk.choices[0].delta, "content", "")
        if response_chunk:
            response += response_chunk
            update_display(Markdown(f"### Refined Query:\n\n{refined_query}\n\n---\n\n### Final Response:\n\n{response}"), display_id=display_handle.display_id)

In [61]:
# Example Usage
question = """
Please explain what this code does:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
ask_with_ollama_and_gpt(raw_question=question)